# VQE in DFT with PsiEmbed and Qiskit

Here we define the inputs as required by PsiEmbed. Note that we'll follow the logic of `embedding_module/run_open_shell`.

We can think of this procedure are requiring three steps:

1. Pre-embedding

    Here we define the problem, and run a low-level calculation of the whole system. From this we obtain the pre-embedded density matrices $\gamma^A$ and $\gamma^B$

    We then define the level-shift projector $P$ and embedding potential $V_{emb}$.

2. Embedding

    Using $V_{emb}$ we run a high-level method simulation of the active region to get the embedded density matrix $\gamma^A_{emb}$.

    We calculate the correction term $tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

3. Post-embedding

    Finally we calculate the embedded energy, by removing $V_{emb}$ from the Hamiltonian, and using density matrix $\gamma^A_{emb}$.

    The total energy is then given by: $E = E[\gamma^A_{emb}] + E[\gamma^B] + g[\gamma^A, \gamma^B] + E_{nuclear} + tr[V_{emb}(\gamma^A_{emb}-\gamma^A)]$

# 0. Set Parameters

First we'll set the parameters 

In [3]:
from copy import copy
from typing import Dict
from spade import fill_defaults
import numpy as np
from scipy import linalg
from spade.main import driver
from spade.embedding_module import run_closed_shell

options = {}
options['geometry'] = """
O          0.00000        0.00000        0.1653507
H          0.00000        0.7493682     -0.4424329
H          0.00000       -0.7493682     -0.4424329
    """
    
options['basis'] = 'STO-6G' # basis set 
options['low_level'] = 'b3lyp' # level of theory of the environment 
options['high_level'] = 'mp2' # level of theory of the embedded system
options['n_active_atoms'] = 1 # number of active atoms (first n atoms in the geometry string)
options['low_level_reference'] = 'rhf'
options['high_level_reference'] = 'rhf'
options['package'] = 'pyscf'

keywords = fill_defaults(options)

e_psiembed = run_closed_shell(keywords)

self._mean_field.e_tot=-76.00109433740889
self._mean_field.e_tot=-52.47023970939136
E with embedded density=-77.90645171911889
e_env=-23.237303494494306, two_e_cross=16.1803718803565, correction=-0.0008673599159010337, embed.nre=9.128283728282938
Final energy=-75.83596696488965
Final energy: -75.83596696488965
e_act_emb=-77.90645171911889, e_env=-23.237303494494306, two_e_cross=16.1803718803565, embed.nre=9.128283728282938, correction=-0.0008673599159010337
e_total=-75.83971289621803


/home/miw/.local/lib/python3.8/site-packages/pyscf/gto/mole.py:1045: UserWarning: Function mol.dumps drops attribute _nelectron because it is not JSON-serializable
  warnings.warn(msg)


# 1. Low-level whole system calculation

The first step is to run a mean field caluclation of the whole system.

The Embed class and its subclasses have a method to do this which also sets the following properties:
    Exchange correlation potentials (v_xc_total if embedding potential is not set, or alpha/beta_v_xc_total)
    

In [4]:
from pyscf import gto, scf, cc, ao2mo, fci

mol = gto.Mole(atom=keywords['geometry'], basis=keywords['basis'], charge=0).build()

ks = scf.RKS(mol)
ks.conv_tol = keywords["e_convergence"]
ks.xc = keywords["low_level"]
e_initial = ks.kernel()

# Store the initial value of h core as this is needed later,
# but is overwritten

initial_h_core = ks.get_hcore()

mol_copy = copy(mol)

converged SCF energy = -76.0010943374089


In [5]:
hf = mol_copy.RHF().run()

ref_fci = fci.FCI(hf)
ref_fci.conv_tol = keywords["e_convergence"]
fci_result = ref_fci.kernel()

expected_energy = fci_result[0]

# What units is this in?
f"{expected_energy=}"

converged SCF energy = -75.680090378264


'expected_energy=-75.73152218912334'

# 2. Orbital Localisation
Find the orbitals of the active space and environment, using SPADE.

In [6]:
n_occupied_orbitals = np.count_nonzero(ks.mo_occ == 2)
occupied_orbitals = ks.mo_coeff[:, :n_occupied_orbitals]

n_act_aos = mol.aoslice_by_atom()[keywords['n_active_atoms']-1][-1]
ao_overlap = ks.get_ovlp()

# Orbital rotation and partition into subsystems A and B
#rotation_matrix, sigma = embed.orbital_rotation(occupied_orbitals,
#    n_act_aos, ao_overlap)

rotated_orbitals = linalg.fractional_matrix_power(ao_overlap, 0.5) @ occupied_orbitals
_, sigma, right_vectors = linalg.svd(rotated_orbitals[:n_act_aos, :])

#n_act_mos, n_env_mos = embed.orbital_partition(sigma)
value_diffs = sigma[1:]-sigma[:-1]
n_act_mos = np.argmin(value_diffs) + 1
n_env_mos = len(sigma) - n_act_mos

# Defining active and environment orbitals and density
act_orbitals = occupied_orbitals @ right_vectors.T[:, :n_act_mos]
env_orbitals = occupied_orbitals @ right_vectors.T[:, n_act_mos:]
act_density = 2.0 * act_orbitals @ act_orbitals.T
env_density = 2.0 * env_orbitals @ env_orbitals.T

# 3. Calculate the cross subsytem terms

In [7]:
# Retrieving the subsytem energy terms and potential matrices

# The function called looks like this
def closed_shell_subsystem(scf, density):
    #It seems that PySCF lumps J and K in the J array 
    j = scf.get_j(dm = density)
    k = np.zeros(np.shape(j))
    two_e_term =  scf.get_veff(scf.mol, density)
    e_xc = two_e_term.exc
    v_xc = two_e_term - j

    # Energy
    e = np.einsum("ij,ij", density, scf.get_hcore() + j/2) + e_xc
    return e, e_xc, j, k, v_xc

e_act, e_xc_act, j_act, k_act, v_xc_act = (
    closed_shell_subsystem(ks, act_density))
e_env, e_xc_env, j_env, k_env, v_xc_env = (
    closed_shell_subsystem(ks, env_density))

# Computing cross subsystem terms
# Note that the matrix dot product is equivalent to the trace.
j_cross = 0.5 * (np.einsum("ij,ij",act_density, j_env)
        + np.einsum("ij,ij", env_density, j_act))

k_cross = 0.0

xc_cross = ks.get_veff().exc - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross
print(f"{e_act=},{e_xc_act=}")#, {j_act=}, {k_act=}, {v_xc_act=}")
f"{two_e_cross=}, {xc_cross=}"

e_act=-78.07244645155401,e_xc_act=-7.237060261886793


'two_e_cross=16.180371880356496, xc_cross=-0.432126222505707'

# 4. Define $V_{emb}$

We can now define the projector used to orthogonalise the Molecular and Atomic orbitals. From this we calculate the embedding potential.

$P_{\alpha, \beta} = S\gamma^BS$

From this we can now also define the embedding potential.

$V_{emb} = g[\gamma^A, \gamma^B] - g[\gamma^A] + \mu P$

In [7]:
# Define the mu-projector
print(f"{keywords['level_shift']=}")
projector = keywords['level_shift'] * (ks.get_ovlp() @ env_density
    @ ks.get_ovlp())

v_xc_total = ks.get_veff() - ks.get_j()

# Defining the embedded core Hamiltonian
v_emb = (j_env + v_xc_total - v_xc_act + projector)

keywords['level_shift']=1000000.0


# 5 Run HF of full system with $V_{emb}$ to get $\gamma^A_{emb}$

Here, PsiEmbed gives us the option to stop, outputting values for calculation by other means.

To continue, we run the mean field method, but with the embedding potentials as calulated.

Note we don't need to run a high-level calculation here as that doesn't change the density matrix.

In [87]:
embedded_scf = scf.RHF(mol)
embedded_scf.conv_tol = keywords["e_convergence"]
embedded_scf.mol.nelectron = 2*n_act_mos

h_core = embedded_scf.get_hcore()

embedded_scf.get_hcore = lambda *args, **kwargs: h_core + v_emb

embedded_scf.kernel()

embedded_occ_orbs = embedded_scf.mo_coeff[:, embedded_scf.mo_occ>0]
embedded_density = 2*embedded_occ_orbs @ embedded_occ_orbs.T

e_emb = embedded_scf.energy_elec(dm=embedded_density, vhf=embedded_scf.get_veff())[0]

print(f"{e_emb=}")

converged SCF energy = -52.4702397094629
e_emb=-61.59852343774585


# 6 Calculate correction term

In [92]:
# Compute the correction
# - There are two versions used for different embeddings
wf_correction = np.einsum("ij,ij", v_emb, embedded_density - act_density)
dft_correction = np.einsum("ij,ij", act_density, v_emb)

print(f"{wf_correction=}, {dft_correction=}")

wf_correction=-0.0008673601610098558, dft_correction=16.308795640885364


# 7 Calculate $E[\gamma^A_{emb}]$

We calculate the Hartree-fock energy of the embedded region, we then add correlation later.

In [115]:
embedded_scf.get_hcore = lambda *args, **kwargs: h_core

# Can use either of these methods 
# This needs to change if we're not using PySCFEmbed
# The j and k matrices are defined differently in PySCF and Psi4
e_act_emb_explicit = np.einsum("ij,ij", embedded_density,  initial_h_core + 0.5 * embedded_scf.get_j() - 0.25 * embedded_scf.get_k())
e_act_emb = embedded_scf.energy_elec(dm=embedded_density, vhf=embedded_scf.get_veff())[0]
print(f"E_HF = {e_act_emb}")
print(f"Difference between HF methods: {e_act_emb - e_act_emb_explicit}")

# Run CCSD as WF method
ccsd = cc.CCSD(embedded_scf)
ccsd.conv_tol = keywords["e_convergence"]

# Set which orbitals are to be frozen
shift = embedded_mol.nao - n_env_mos
fos = [i for i in range(shift, embedded_mol.nao)]
ccsd.frozen = fos

ccsd.run()
correlation = ccsd.e_corr
e_act_emb += correlation

f"{e_act_emb=}"

E_HF = -77.90645171903793
Difference between HF methods: 2.842170943040401e-14
E(CCSD) = -68.78827982029462  E_corr = -0.01011182953962096


'e_act_emb=-77.91656354857756'

# 8 Add all the parts up.

e_act_emb : $\epsilon[\gamma^A_{emb}]$
>energy of the embedded region

e_env : $E[\gamma^B]$ 
>energy of the environment

two_e_cross : $g[\gamma^A, \gamma^B]$
>non-additive two electron term

embed.nre
>The Coulomb energy from nuclear repulsion.

correction : $tr[(\gamma^A_{emb} - \gamma^A)(h^{A in B} - h)]$ (or $tr[\gamma^A(h^{A in B} - h)]$ )
> Correction for embedding

In [116]:
e_nuc = mol.energy_nuc()

e_mf_emb = e_act_emb + e_env + two_e_cross + wf_correction + e_nuc
print("Component contributions")
print(f"{e_act_emb=}, {e_env=}, {two_e_cross=}, {wf_correction=}, {e_nuc=}\n")

# Print out the final value.
print(f"Full system low-level Energy:\t{e_initial}")
print(f"FCI Energy:\t\t\t{expected_energy}")
print(f"Embedding Energy:\t\t{e_mf_emb}")
print(f"Error:\t\t\t\t{(expected_energy-e_mf_emb)*100/expected_energy:.2f}%")


Component contributions
e_act_emb=-77.91656354857756, e_env=-23.237303494494313, two_e_cross=16.180371880356496, wf_correction=-0.0008673601610098558, e_nuc=9.128283728282938

Full system low-level Energy:	-76.00109433740889
FCI Energy:			-75.73152218912334
Embedding Energy:		-75.84607879459345
Error:				-0.15%
